# Anexo 7: Bosque Aleatorio

En este notebook se realizará la creación del modelo de Bosque Aleatorio, siguiendo los pasos definidos en la documentación para la creación de modelos de clasificación

In [1]:
import pandas as pd
import numpy as np
import feather
import matplotlib as mpl
mpl.use('nbAgg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import label_binarize
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

In [2]:
pingfInfoFilePath = './data/dv_personas_190411_abt'
dv_personas_190411_model = pd.read_feather(pingfInfoFilePath,columns=None, use_threads=True)
dv_personas_190411_model.head()

,sexo,tipo_acceso,tipo_erasmus,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),...,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN HUMANIDADES,Titulacion_is_MÁSTER EN GESTIÓN DEL CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA.,Titulacion_is_MÁSTER EN GESTIÓN Y DIRECCIÓN HOSTELERA,Titulacion_is_MÁSTER UNIVERSITARIO EN FORMACIÓN DEL PROFESORADO Y TIC,Titulacion_is_PCEO: DERECHO / ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS / DERECHO,Titulacion_is_PCEO: ADMINISTRACIÓN Y DIRECCIÓN DE EMPRESAS /CIENCIAS DEL TRABAJO,Titulacion_is_MÁSTER UNIVERSITARIO GESTIÓN CONOCIMIENTO BIOMÉDICO E INVESTIGACIÓN CLÍNICA,Titulacion_is_MÁSTER UNIVERSITARIO EN INVESTIGACIÓN EN CIENCIAS SOCIOSANITARIAS,Titulacion_is_MÁSTER UNIVERSITARIO EN PSICOPEDAGOGÍA
0,M,Selectividad,Ninguno,21,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M,Selectividad,Ninguno,26,28.0,2ec47253d8c59fa32f000519d2f9b83c26c7f11dcc1ed8...,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,M,Acceso a Segundo Ciclo,Ninguno,24,21.0,2d2f9c8a7e3c780fef578a2be45f60d537480aeaf01a88...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,H,Selectividad,Ninguno,21,37.0,df8a544bdeb3b60dfe3c5ab2abce943b2e8769ca4693ce...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M,Título Universitario,Ninguno,32,78.0,0b7cafeb91821d6c39ccc2f3058edd9c77f351f0047f1a...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Creación de variables ficticias

#### Se procede a crear las variables ficticias de las diferentes variables categóricas del dataset

In [3]:
categoricalCols = ['sexo','tipo_acceso','tipo_erasmus']

In [4]:
dv_personas_190411_model_one_hot = pd.get_dummies(data=dv_personas_190411_model,columns=categoricalCols)

In [5]:
dv_personas_190411_model_one_hot.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,dni,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,23.0,0013670b45b4207d01539bc1d057daa03bab0e2de19916...,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,28.0,2ec47253d8c59fa32f000519d2f9b83c26c7f11dcc1ed8...,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,21.0,2d2f9c8a7e3c780fef578a2be45f60d537480aeaf01a88...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,37.0,df8a544bdeb3b60dfe3c5ab2abce943b2e8769ca4693ce...,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,78.0,0b7cafeb91821d6c39ccc2f3058edd9c77f351f0047f1a...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [6]:
dv_personas_190411_model_one_hot.drop(['dni'],axis=1,inplace = True)

### Agrupación de los datos de la variable objetivo en bloques de 33'333%

In [7]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,4)
tto_pcto_dias_trabajados_bucket_array

array([  0.        ,  33.33333333,  66.66666667, 100.        ])

In [8]:
dv_personas_190411_model_one_hot_33 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,"(-0.001, 33.333]",1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,"(-0.001, 33.333]",0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,"(-0.001, 33.333]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,"(33.333, 66.667]",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,"(66.667, 100.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [9]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 33.333]
1        (-0.001, 33.333]
2        (-0.001, 33.333]
3        (33.333, 66.667]
4         (66.667, 100.0]
               ...       
14315    (-0.001, 33.333]
14316    (-0.001, 33.333]
14317    (-0.001, 33.333]
14318    (-0.001, 33.333]
14319    (-0.001, 33.333]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (3, interval[float64]): [(-0.001, 33.333] < (33.333, 66.667] < (66.667, 100.0]]

In [10]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_33.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_33.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [11]:
dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados'].value_counts()

0    9286
1    4013
2    1021
Name: tto_pcto_dias_trabajados, dtype: int64

### División de los datos en características y variable objetivo

In [12]:
Features_1_2 = dv_personas_190411_model_one_hot_33.iloc[:,0:1]
Features_2_2 = dv_personas_190411_model_one_hot_33.iloc[:,2:]
Labels = dv_personas_190411_model_one_hot_33['tto_pcto_dias_trabajados']

In [13]:
Features = np.array(Features_1_2)
Features = np.concatenate([Features, np.array(Features_2_2)], axis = 1)
Features[:2,:]

array([[21,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0],
       [26,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [14]:
Labels = np.array(Labels)
Labels[:20]

array([0, 0, 0, 1, 2, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 0, 1, 2, 2],
      dtype=int8)

In [30]:
nr.seed(123)
inside = ms.KFold(n_splits=3, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=10, shuffle = True)

### Búsueda de mejores híper-parámetros para el modelo


In [49]:
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'min_samples_split': [2, 5, 10, 15, 100],
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10,20],
    'criterion' : ['gini', 'entropy'],
    'max_features' : ['auto', 'sqrt', 'log2']
}
## Define the random forest model
nr.seed(3456)
rf_clf_33 = RandomForestClassifier(class_weight = "balanced") # class_weight = {0:0.33, 1:0.67}) 

## Perform the grid search over the parameters
nr.seed(4455)
rf_clf_33 = ms.GridSearchCV(estimator = rf_clf_33, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)
rf_clf_33.fit(Features, Labels)
print(rf_clf_33.best_estimator_.n_estimators)
print(rf_clf_33.best_estimator_.min_samples_leaf)
print(rf_clf_33.best_estimator_.max_features)
print(rf_clf_33.best_estimator_.min_samples_split)
print(rf_clf_33.best_estimator_.criterion)
print(rf_clf_33.best_estimator_.max_depth)


300
1
auto
100
entropy
30


### Evaluación del modelo a través del método de validación cruzada

In [16]:
nr.seed(498)
cv_estimate = ms.cross_val_score(rf_clf_33, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.671
SDT of the metric       = 0.006
Outcomes by cv fold
Fold  1    0.679
Fold  2    0.666
Fold  3    0.669


In [31]:
rf_clf_33 = RandomForestClassifier(class_weight = "balanced",
                                n_estimators = 300,
                                min_samples_leaf = 1,
                                max_features = 'auto',
                                min_samples_split = 100,
                                criterion = 'entropy',
                                max_depth = 30
                               ) 
nr.seed(498)
cv_estimate = ms.cross_val_score(rf_clf_33, Features, Labels, 
                                 cv = outside, scoring = 'accuracy') # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.528
SDT of the metric       = 0.007
Outcomes by cv fold
Fold  1    0.540
Fold  2    0.527
Fold  3    0.527
Fold  4    0.530
Fold  5    0.521
Fold  6    0.520
Fold  7    0.520
Fold  8    0.525
Fold  9    0.536
Fold 10    0.534


### División de los datos en datos de prueba y entrenamiento

In [20]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=45)

### Entrenamiento del modelo

In [23]:
nr.seed(1115)
rf_mod_33 = RandomForestClassifier(class_weight = "balanced",
                                n_estimators = 300,
                                min_samples_leaf = 1,
                                max_features = 'auto',
                                min_samples_split = 100,
                                criterion = 'entropy',
                                max_depth = 30
                               ) 
rf_mod_33.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
scores = rf_mod_33.predict(x_test)

### Evaluación del modelo

In [26]:
def print_metrics_33(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Actual [0,333.333]        %6d' % conf[0,0] + '             %5d' % conf[0,1] + '             %5d' % conf[0,2])
    print('Actual (33.333,66.667]    %6d' % conf[1,0] + '             %5d' % conf[1,1] + '             %5d' % conf[1,2])
    print('Actual (66.667,100]       %6d' % conf[2,0] + '             %5d' % conf[2,1] + '             %5d' % conf[2,2])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]')
    print('Num case   %0.2f' % metrics[3][0] + '                  %0.2f' % metrics[3][1] + '                  %0.2f' % metrics[3][2] )
    print('Precision   %0.2f' % metrics[0][0] + '                      %0.2f' % metrics[0][1] + '                   %0.2f' % metrics[0][2] )
    print('Recall      %0.2f' % metrics[1][0] + '                      %0.2f' % metrics[1][1] + '                   %0.2f' % metrics[1][2] )
    print('F1          %0.2f' % metrics[2][0] + '                      %0.2f' % metrics[2][1] + '                   %0.2f' % metrics[2][2] )

In [27]:
print_metrics_33(y_test, scores)  

                 Confusion matrix
                 Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Actual [0,333.333]          1448               485               396
Actual (33.333,66.667]       394               300               286
Actual (66.667,100]           60                70               141

Accuracy        0.53
 
         Score [0,333.333]    Score (33.333,66.667] Score (66.667,100]
Num case   2329.00                  980.00                  271.00
Precision   0.76                      0.35                   0.17
Recall      0.62                      0.31                   0.52
F1          0.68                      0.33                   0.26


### Agrupación de los datos de la variable objetivo en bloques de 25%

In [24]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,5)
tto_pcto_dias_trabajados_bucket_array

array([  0.,  25.,  50.,  75., 100.])

In [25]:
dv_personas_190411_model_one_hot_25 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_25.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,"(-0.001, 25.0]",1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,"(25.0, 50.0]",0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,"(-0.001, 25.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,"(25.0, 50.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,"(75.0, 100.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [26]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'].value_counts()

(-0.001, 25.0]    7497
(25.0, 50.0]      4319
(50.0, 75.0]      1966
(75.0, 100.0]      538
Name: tto_pcto_dias_trabajados, dtype: int64

In [27]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'].astype('category')

0        (-0.001, 25.0]
1          (25.0, 50.0]
2        (-0.001, 25.0]
3          (25.0, 50.0]
4         (75.0, 100.0]
              ...      
14315    (-0.001, 25.0]
14316    (-0.001, 25.0]
14317    (-0.001, 25.0]
14318    (-0.001, 25.0]
14319    (-0.001, 25.0]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (4, interval[float64]): [(-0.001, 25.0] < (25.0, 50.0] < (50.0, 75.0] < (75.0, 100.0]]

In [28]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_25.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_25.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [29]:
dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados'].value_counts()

0    7497
1    4319
2    1966
3     538
Name: tto_pcto_dias_trabajados, dtype: int64

### División de los datos en características y variable objetivo

In [30]:
Features_1_2 = dv_personas_190411_model_one_hot_25.iloc[:,0:1]
Features_2_2 = dv_personas_190411_model_one_hot_25.iloc[:,2:]
Labels = dv_personas_190411_model_one_hot_25['tto_pcto_dias_trabajados']

In [31]:
Features = np.array(Features_1_2)
Features = np.concatenate([Features, np.array(Features_2_2)], axis = 1)
Features[:2,:]

array([[21,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0],
       [26,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [32]:
Labels = np.array(Labels)
Labels[:20]


array([0, 1, 0, 1, 3, 0, 1, 3, 0, 2, 2, 1, 1, 0, 0, 3, 0, 1, 2, 3],
      dtype=int8)

In [38]:
nr.seed(123)
inside = ms.KFold(n_splits=3, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=3, shuffle = True)

### Búsueda de mejores híper-parámetros para el modelo

In [27]:
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'min_samples_split': [2, 5, 10, 15, 100],
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10,20],
    'criterion' : ['gini', 'entropy'],
    'max_features' : ['auto', 'sqrt', 'log2']
}
## Define the random forest model
nr.seed(3456)
rf_clf_25 = RandomForestClassifier(class_weight = "balanced") # class_weight = {0:0.33, 1:0.67}) 

## Perform the grid search over the parameters
nr.seed(4455)
rf_clf_25 = ms.GridSearchCV(estimator = rf_clf_25, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)
rf_clf_25.fit(Features, Labels)
print(rf_clf_25.best_estimator_.n_estimators)
print(rf_clf_25.best_estimator_.min_samples_leaf)
print(rf_clf_25.best_estimator_.max_features)
print(rf_clf_25.best_estimator_.min_samples_split)
print(rf_clf_25.best_estimator_.criterion)
print(rf_clf_25.best_estimator_.max_depth)



800
5
log2
10
gini
30


### Evaluación del modelo a través del método de validación cruzada

In [39]:
nr.seed(498)
cv_estimate = ms.cross_val_score(rf_clf_25, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.666
SDT of the metric       = 0.011
Outcomes by cv fold
Fold  1    0.680
Fold  2    0.653
Fold  3    0.664


### División de los datos en datos de prueba y entrenamiento

In [33]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=42)

### Entrenamiento del modelo

In [34]:
nr.seed(1115)
rf_mod_25 = RandomForestClassifier(class_weight = "balanced",
                                n_estimators = 800,
                                min_samples_leaf = 5,
                                max_features = 'log2',
                                min_samples_split = 10,
                                criterion = 'gini',
                                max_depth = 30
                               ) 
rf_mod_25.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
scores = rf_mod_25.predict(x_test)

### Evaluación del modelo


In [36]:
def print_metrics_25(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,25]    Score (25,50]   Score (50,75]   Score (75,100]')
    print('Actual [0,25]     %6d' % conf[0,0] + '             %5d' % conf[0,1] + '        %5d' % conf[0,2] + '          %5d' % conf[0,3])
    print('Actual (25,50]    %6d' % conf[1,0] + '             %5d' % conf[1,1] + '        %5d' % conf[1,2] + '          %5d' % conf[1,3])
    print('Actual (50,75]    %6d' % conf[2,0] + '             %5d' % conf[2,1] + '        %5d' % conf[2,2] + '          %5d' % conf[2,3])
    print('Actual (75,100]   %6d' % conf[3,0] + '             %5d' % conf[3,1] + '        %5d' % conf[3,2] + '          %5d' % conf[3,3])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('          Score [0,25]  Score (25,50]  Score (50,75] Score (75,100]')
    print('Num case   %0.2f' % metrics[3][0] + '         %0.2f' % metrics[3][1] + '        %0.2f' % metrics[3][2] + '       %0.2f' % metrics[3][3] )
    print('Precision   %0.2f' % metrics[0][0] + '            %0.2f' % metrics[0][1] + '           %0.2f' % metrics[0][2] + '         %0.2f' % metrics[0][3] )
    print('Recall      %0.2f' % metrics[1][0] + '            %0.2f' % metrics[1][1] + '           %0.2f' % metrics[1][2] + '         %0.2f' % metrics[1][3] )
    print('F1          %0.2f' % metrics[2][0] + '            %0.2f' % metrics[2][1] + '           %0.2f' % metrics[2][2] + '         %0.2f' % metrics[2][3] )

In [37]:
print_metrics_25(y_test, scores)  

                 Confusion matrix
                 Score [0,25]    Score (25,50]   Score (50,75]   Score (75,100]
Actual [0,25]        969               339          127            394
Actual (25,50]       439               261          123            295
Actual (50,75]       138                75           74            215
Actual (75,100]       20                15           11             85

Accuracy        0.39
 
          Score [0,25]  Score (25,50]  Score (50,75] Score (75,100]
Num case   1829.00         1118.00        502.00       131.00
Precision   0.62            0.38           0.22         0.09
Recall      0.53            0.23           0.15         0.65
F1          0.57            0.29           0.18         0.15


#### Procedemos a dividir el tto_pcto_dias_trabajados en bloques de 20

In [40]:
tto_pcto_dias_trabajados_bucket_array = np.linspace(0,100,6)
tto_pcto_dias_trabajados_bucket_array

array([  0.,  20.,  40.,  60.,  80., 100.])

In [41]:
dv_personas_190411_model_one_hot_20 = dv_personas_190411_model_one_hot.copy()
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'] = pd.cut(dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'],tto_pcto_dias_trabajados_bucket_array,include_lowest=True)
dv_personas_190411_model_one_hot_20.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,"(20.0, 40.0]",1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,"(20.0, 40.0]",0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,"(20.0, 40.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,"(20.0, 40.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,"(60.0, 80.0]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [42]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'].astype('category')

0          (20.0, 40.0]
1          (20.0, 40.0]
2          (20.0, 40.0]
3          (20.0, 40.0]
4          (60.0, 80.0]
              ...      
14315    (-0.001, 20.0]
14316    (-0.001, 20.0]
14317    (-0.001, 20.0]
14318      (20.0, 40.0]
14319    (-0.001, 20.0]
Name: tto_pcto_dias_trabajados, Length: 14320, dtype: category
Categories (5, interval[float64]): [(-0.001, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]

In [43]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'] = dv_personas_190411_model_one_hot_20.tto_pcto_dias_trabajados.cat.codes
dv_personas_190411_model_one_hot_20.head()

,edad_fin_primera_titulacion,tto_pcto_dias_trabajados,Titulacion_is_GRADO EN EDUCACIÓN PRIMARIA,Titulacion_is_ING. TÉC. OBRAS PÚB. (ESPECIALIDAD EN CONST. CIV.),Titulacion_is_INGENIERO QUÍMICO,Titulacion_is_INGENIERO TÉCNICO FORESTAL (ESPECIALIDAD EN EXPLOTACIONES FORESTALES),Titulacion_is_GRADO EN MEDICINA,Titulacion_is_INGENIERO TÉCNICO INDUSTRIAL (ESPECIALIDAD EN ELECTRICIDAD),Titulacion_is_DIPLOMADO EN ENFERMERÍA,Titulacion_is_DIPLOMADO EN TERAPIA OCUPACIONAL,...,tipo_acceso_Estudios universitarios extranjeros parcialmente convalidados,tipo_acceso_Formación Profesional,tipo_acceso_Mayores de 25/40/45 años,tipo_acceso_Otros accesos a primer y segundo ciclo,tipo_acceso_Selectividad,tipo_acceso_Traslado de Expediente (Estudios Españoles),tipo_acceso_Título Universitario,tipo_erasmus_Estudios,tipo_erasmus_Ninguno,tipo_erasmus_Prácticas
0,21,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,26,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,24,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,21,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
4,32,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [44]:
dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados'].value_counts()

0    6184
1    4261
2    2374
3    1127
4     374
Name: tto_pcto_dias_trabajados, dtype: int64

In [45]:
Features_1_2 = dv_personas_190411_model_one_hot_20.iloc[:,0:1]
Features_2_2 = dv_personas_190411_model_one_hot_20.iloc[:,2:]
Labels = dv_personas_190411_model_one_hot_20['tto_pcto_dias_trabajados']

In [46]:
Features = np.array(Features_1_2)
Features = np.concatenate([Features, np.array(Features_2_2)], axis = 1)
Features[:2,:]

array([[21,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  1,  0],
       [26,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       

In [47]:
Labels = np.array(Labels)
Labels[:20]

array([1, 1, 1, 1, 3, 0, 1, 3, 0, 3, 3, 2, 2, 0, 0, 3, 0, 2, 3, 3],
      dtype=int8)

In [88]:
nr.seed(123)
inside = ms.KFold(n_splits=3, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=3, shuffle = True)

In [89]:
param_grid = {
    'n_estimators': [100, 300, 500, 800, 1200],
    'min_samples_split': [2, 5, 10, 15, 100],
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10,20],
    'criterion' : ['gini', 'entropy'],
    'max_features' : ['auto', 'sqrt', 'log2']
}
## Define the random forest model
nr.seed(3456)
rf_clf_20 = RandomForestClassifier(class_weight = "balanced") # class_weight = {0:0.33, 1:0.67}) 

## Perform the grid search over the parameters
nr.seed(4455)
rf_clf_20 = ms.GridSearchCV(estimator = rf_clf_20, param_grid = param_grid, 
                      cv = inside, # Use the inside folds
                      scoring = 'roc_auc_ovo_weighted',
                      return_train_score = True)
rf_clf_20.fit(Features, Labels)
print(rf_clf_20.best_estimator_.n_estimators)
print(rf_clf_20.best_estimator_.min_samples_leaf)
print(rf_clf_20.best_estimator_.max_features)
print(rf_clf_20.best_estimator_.min_samples_split)
print(rf_clf_20.best_estimator_.criterion)
print(rf_clf_20.best_estimator_.max_depth)

800
2
auto
2
entropy
15


In [90]:
nr.seed(498)
cv_estimate = ms.cross_val_score(rf_clf_20, Features, Labels, 
                                 cv = outside) # Use the outside folds

print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
print('Outcomes by cv fold')
for i, x in enumerate(cv_estimate):
    print('Fold %2d    %4.3f' % (i+1, x))

Mean performance metric = 0.651
SDT of the metric       = 0.007
Outcomes by cv fold
Fold  1    0.661
Fold  2    0.649
Fold  3    0.644


In [48]:
x_train, x_test, y_train, y_test = ms.train_test_split(Features,Labels,test_size=0.25, random_state=42)

In [92]:
nr.seed(1115)
rf_mod_20 = RandomForestClassifier(class_weight = "balanced",
                                n_estimators = rf_clf_20.best_estimator_.n_estimators,
                                min_samples_leaf = rf_clf_20.best_estimator_.min_samples_leaf,
                                max_features = rf_clf_20.best_estimator_.max_features,
                                min_samples_split = rf_clf_20.best_estimator_.min_samples_split,
                                criterion = rf_clf_20.best_estimator_.criterion,
                                max_depth = rf_clf_20.best_estimator_.max_depth
                               ) 
rf_mod_20.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
nr.seed(1115)
rf_mod_20 = RandomForestClassifier(class_weight = "balanced",
                                n_estimators = 800,
                                min_samples_leaf = 2,
                                max_features = 'auto',
                                min_samples_split = 2,
                                criterion = 'entropy',
                                max_depth = 15
                               ) 
rf_mod_20.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [50]:
scores = rf_mod_20.predict(x_test)

In [51]:
def print_metrics_20(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Score [0,20]    Score (20,40] Score (40,60] Score (60,80] Score (80,100]')
    print('Actual [0,20]     %6d' % conf[0,0] + '           %5d' % conf[1,1] + '            %5d' % conf[1,2] + '         %5d' % conf[1,3] + '        %5d' % conf[1,4])
    print('Actual (40,60]   %6d' % conf[2,0] + '            %5d' % conf[2,1] + '            %5d' % conf[2,2] + '         %5d' % conf[2,3] + '        %5d' % conf[2,4])
    print('Actual (60,80]   %6d' % conf[3,0] + '            %5d' % conf[3,1] + '            %5d' % conf[3,2] + '         %5d' % conf[3,3] + '        %5d' % conf[3,4])
    print('Actual (80,100]  %6d' % conf[4,0] + '            %5d' % conf[4,1] + '            %5d' % conf[4,2] + '         %5d' % conf[4,3] + '        %5d' % conf[4,4])

    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('          Score [0,20]  Score (20,40]  Score (40,60]  Score (60,80]  Score (80,100]')
    print('Num case   %0.2f' % metrics[3][0] + '         %0.2f' % metrics[3][1] + '         %0.2f' % metrics[3][2] + '          %0.2f' % metrics[3][3] + '       %0.2f' % metrics[3][4])
    print('Precision   %0.2f' % metrics[0][0] + '            %0.2f' % metrics[0][1] + '            %0.2f' % metrics[0][2] + '            %0.2f' % metrics[0][3] + '         %0.2f' % metrics[0][4])
    print('Recall      %0.2f' % metrics[1][0] + '            %0.2f' % metrics[1][1] + '            %0.2f' % metrics[1][2] + '            %0.2f' % metrics[1][3] + '         %0.2f' % metrics[1][4])
    print('F1          %0.2f' % metrics[2][0] + '            %0.2f' % metrics[2][1] + '            %0.2f' % metrics[2][2] + '            %0.2f' % metrics[2][3] + '         %0.2f' % metrics[2][4])

In [52]:
print_metrics_20(y_test, scores)  

                 Confusion matrix
                 Score [0,20]    Score (20,40] Score (40,60] Score (60,80] Score (80,100]
Actual [0,20]        791             260              108            65          185
Actual (40,60]      221              102               65            66          165
Actual (60,80]       77               22               26            40          132
Actual (80,100]      25                2                5             8           45

Accuracy        0.34
 
          Score [0,20]  Score (20,40]  Score (40,60]  Score (60,80]  Score (80,100]
Num case   1522.00         1057.00         619.00          297.00       85.00
Precision   0.51            0.37            0.22            0.15         0.06
Recall      0.52            0.25            0.11            0.13         0.53
F1          0.51            0.30            0.14            0.14         0.11


In [100]:
probs = rf_mod_20.predict_proba(x_test)

In [101]:
sklm.roc_auc_score(y_test,probs,multi_class='ovo',average='weighted')

0.6401495186482331